In [2]:
# !pip install emoji
# !pip install transformers

In [3]:
import pandas as pd
import numpy as np
import re
from random import random
import emoji
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from transformers import RobertaTokenizer, RobertaModel
import json

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

## Loading data

In [6]:
with open('evaluation/pubilc_data/train.json',encoding="utf-8") as f:
    train_data = json.load(f)
with open('evaluation/pubilc_data/dev.json',encoding="utf-8") as g:
    dev_data = json.load(g)
with open('evaluation/pubilc_data/eval.json',encoding="utf-8") as h:
    test_data = json.load(h)
df_train = pd.DataFrame(train_data)
df_dev = pd.DataFrame(dev_data)
df_test = pd.DataFrame(test_data)

print("Number of text in training data: {}".format(df_train.shape[0]))
# print("Number of text in categories: {}".format(categories_type.shape[1]))
print("Number of text in developing data: {}".format(df_dev.shape[0]))
print("Number of text in testing data: {}".format(df_test.shape[0]))

Number of text in training data: 168521
Number of text in developing data: 40487
Number of text in testing data: 88664


In [7]:
# df_train = pd.read_json('/content/drive/Shareddrives/NLP/source/train.json', lines=True)
# # categories_type = pd.read_json('./source/categories.json', lines=True)
# df_dev = pd.read_json('/content/drive/Shareddrives/NLP/source/dev.json', lines=True)
# df_test = pd.read_json('/content/drive/Shareddrives/NLP/source/eval.json', lines=True)
# print("Number of text in training data: {}".format(df_train.shape[0]))
# # print("Number of text in categories: {}".format(categories_type.shape[1]))
# print("Number of text in developing data: {}".format(df_dev.shape[0]))
# print("Number of text in testing data: {}".format(df_test.shape[0]))

In [8]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# Output tokenizer vocab
#./roberta_vocab/ -> /content/drive/Shareddrives/NLP/roberta_vocab/

tokenizer.save_vocabulary('NLP/roberta_vocab/')

Vocabulary path (NLP/roberta_vocab/) should be a directory


In [10]:
roberta_vocab = pd.read_json('roberta_vocab/vocab.json', typ='series')

In [11]:
list(roberta_vocab.keys())

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 '.',
 'Ġthe',
 ',',
 'Ġto',
 'Ġand',
 'Ġof',
 'Ġa',
 'Ġin',
 '-',
 'Ġfor',
 'Ġthat',
 'Ġon',
 'Ġis',
 'âĢ',
 "'s",
 'Ġwith',
 'ĠThe',
 'Ġwas',
 'Ġ"',
 'Ġat',
 'Ġit',
 'Ġas',
 'Ġsaid',
 'Ļ',
 'Ġbe',
 's',
 'Ġby',
 'Ġfrom',
 'Ġare',
 'Ġhave',
 'Ġhas',
 ':',
 'Ġ(',
 'Ġhe',
 'ĠI',
 'Ġhis',
 'Ġwill',
 'Ġan',
 'Ġthis',
 ')',
 'ĠâĢ',
 'Ġnot',
 'Ŀ',
 'Ġyou',
 'ľ',
 'Ġtheir',
 'Ġor',
 'Ġthey',
 'Ġwe',
 'Ġbut',
 'Ġwho',
 'Ġmore',
 'Ġhad',
 'Ġbeen',
 'Ġwere',
 'Ġabout',
 ',"',
 'Ġwhich',
 'Ġup',
 'Ġits',
 'Ġcan',
 'Ġone',
 'Ġout',
 'Ġalso',
 'Ġ$',
 'Ġher',
 'Ġall',
 'Ġafter',
 '."',
 '/',
 'Ġwould',
 "'t",
 'Ġyear',
 'Ġwhen',
 'Ġfirst',
 'Ġshe',
 'Ġtwo',
 'Ġover',
 'Ġpeople',
 'ĠA',
 'Ġour',
 'ĠIt',
 'Ġtime',
 'Ġthan',
 'Ġinto',
 'Ġthere',
 't',
 'ĠHe',
 'Ġnew',
 'ĠâĢĶ',
 'Ġlast',
 'Ġjust',
 'ĠIn',
 'Ġother',
 'Ġso',
 'Ġwhat',
 'I',
 'Ġlike',
 'a',
 'Ġsome',
 'S',
 'Ã«',
 'Ġthem',
 'Ġyears',
 "'",
 'Ġdo',
 'Ġyour',
 'Ġ-',
 'Ġ1',
 '"',
 'Ġif',
 'Ġcould',
 '?',

## Start analyze

In [12]:
def get_vocab(corpus):
    vocabulary = Counter()
    for sentence in corpus:
        for word in sentence.split():
            vocabulary.update([word])
    return vocabulary

In [13]:
def check_coverage(vocabs, roberta_vocab):
    known_words = {}
    unknown_words = {}
    known_count = 0
    unknown_count = 0
    for word in tqdm(vocabs.keys(), desc='Checking: '):
        if word in list(roberta_vocab.keys()):
            known_words[word] = roberta_vocab[word]
            known_count += vocabs[word]
        elif 'Ġ' + word in list(roberta_vocab.keys()):
            # If we have deep look in roberta tokenizer, many words combine with 'Ġ' since roberta is byte pair encoding
            known_words[word] = roberta_vocab['Ġ' + word]
            known_count += vocabs[word]            
        else:
            unknown_words[word] = vocabs[word]
            unknown_count += vocabs[word]
    print("Found embeddings for {:.3%} ({} / {}) of vocab".format(len(known_words) / len(vocabs), len(known_words), len(vocabs)))
    print("Found embeddings for {:.3%} ({} / {}) of all text".format(known_count / (known_count + unknown_count), known_count, (known_count + unknown_count)))
    return unknown_words

In [14]:
train_text_vocab = get_vocab(df_train['text'].values)
train_reply_vocab = get_vocab(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
# unknown_text = check_coverage(train_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 84676
train reply unique vocab count is: 243082


In [15]:
dev_text_vocab = get_vocab(df_dev['text'].values)
dev_reply_vocab = get_vocab(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
# unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 33864
dev reply unique vocab count is: 85467


In [16]:
test_text_vocab = get_vocab(df_test['text'].values)
test_reply_vocab = get_vocab(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
# unknown_text = check_coverage(test_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 18380
test reply unique vocab count is: 158246


### Try to convert to lower case

In [17]:
def get_vocab_lower(corpus):
    vocabulary = Counter()
    for sentance in corpus:
        for word in sentance.lower().split():
            vocabulary.update([word])
    return vocabulary

In [18]:
train_text_vocab = get_vocab_lower(df_train['text'].values)
train_reply_vocab = get_vocab_lower(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
# unknown_text_lower = check_coverage(train_text_vocab, roberta_vocab)
# print()
# unknown_reply_lower = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 74530
train reply unique vocab count is: 216697


In [19]:
dev_text_vocab = get_vocab_lower(df_dev['text'].values)
dev_reply_vocab = get_vocab_lower(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
# unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 30081
dev reply unique vocab count is: 76417


In [20]:
test_text_vocab = get_vocab_lower(df_test['text'].values)
test_reply_vocab = get_vocab_lower(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
# unknown_text = check_coverage(test_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 16587
test reply unique vocab count is: 141585


### Show some unknown tokens!

In [20]:
# unknown_text

## Clean weird punctuations

#### No significantly improve

In [21]:
def clean_weird(text):
    specials = ["’", "‘", "´", "`"]
    text = text.replace("’", "'")
    text = text.replace("‘", "'")
    text = text.replace("´", "'")
    text = text.replace("`", "'")
    return text

In [22]:
df_train['text'] = df_train.text.apply(clean_weird)
df_train['reply'] = df_train.reply.apply(clean_weird)

In [23]:
df_dev['text'] = df_dev.text.apply(clean_weird)
df_dev['reply'] = df_dev.reply.apply(clean_weird)

In [24]:
df_test['text'] = df_test.text.apply(clean_weird)
df_test['reply'] = df_test.reply.apply(clean_weird)

In [25]:
train_text_vocab = get_vocab(df_train['text'].values)
train_reply_vocab = get_vocab(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
# unknown_text = check_coverage(train_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 84372
train reply unique vocab count is: 242259


In [26]:
dev_text_vocab = get_vocab(df_dev['text'].values)
dev_reply_vocab = get_vocab(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
# unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 33733
dev reply unique vocab count is: 85206


In [27]:
test_text_vocab = get_vocab(df_test['text'].values)
test_reply_vocab = get_vocab(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
# unknown_text = check_coverage(test_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 18306
test reply unique vocab count is: 157848


## Transform apostrophes

In [28]:
apostrophes = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [29]:
def change_apostrophes(text):
    # Replace apostrophes to original term
    for key in apostrophes.keys():
        text = text.replace(key, apostrophes[key])
    return text

In [30]:
df_train['text'] = df_train.text.apply(change_apostrophes)
df_train['reply'] = df_train.reply.apply(change_apostrophes)

In [31]:
df_dev['text'] = df_dev.text.apply(change_apostrophes)
df_dev['reply'] = df_dev.reply.apply(change_apostrophes)

In [32]:
df_test['text'] = df_test.text.apply(change_apostrophes)
df_test['reply'] = df_test.reply.apply(change_apostrophes)

In [33]:
train_text_vocab = get_vocab(df_train['text'].values)
train_reply_vocab = get_vocab(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
# unknown_text = check_coverage(train_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 84203
train reply unique vocab count is: 241990


In [34]:
dev_text_vocab = get_vocab(df_dev['text'].values)
dev_reply_vocab = get_vocab(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
# unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 33645
dev reply unique vocab count is: 85077


In [35]:
test_text_vocab = get_vocab(df_test['text'].values)
test_reply_vocab = get_vocab(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
# unknown_text = check_coverage(test_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 18241
test reply unique vocab count is: 157666


In [ ]:
# unknown_text

NameError: ignored

## Check punctuations that roberta unknown 

In [36]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def unknown_punctuation(roberta_vocab):
    unknown = ''
    for char in punct:
        if char not in list(roberta_vocab.keys()):
            unknown += char
            unknown += ' '
    return unknown

In [37]:
print("Roberta unknown: ")
print(unknown_punctuation(roberta_vocab))

Roberta unknown: 
“ ” ’ ∞ θ α • − β ∅ π ‘ ₹ € ™ √ — – 


### Mapping unknown to known punctuations

In [38]:
def change_punc(text):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    for p in punct:
        text = text.replace(p, ' {} '.format(p))
    return text

In [39]:
df_train['map_punc_text'] = df_train.text.apply(change_punc)
df_train['map_punc_reply'] = df_train.reply.apply(change_punc)

In [40]:
df_dev['map_punc_text'] = df_dev.text.apply(change_punc)
df_dev['map_punc_reply'] = df_dev.reply.apply(change_punc)

In [41]:
df_test['map_punc_text'] = df_test.text.apply(change_punc)
df_test['map_punc_reply'] = df_test.reply.apply(change_punc)

In [42]:
train_text_vocab = get_vocab(df_train['map_punc_text'].values)
train_reply_vocab = get_vocab(df_train['map_punc_reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
print()

# unknown_text = check_coverage(train_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 56490
train reply unique vocab count is: 156122



In [43]:
dev_text_vocab = get_vocab(df_dev['map_punc_text'].values)
dev_reply_vocab = get_vocab(df_dev['map_punc_reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
# unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 24399
dev reply unique vocab count is: 58692


In [44]:
test_text_vocab = get_vocab(df_test['map_punc_text'].values)
test_reply_vocab = get_vocab(df_test['map_punc_reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
# unknown_text = check_coverage(test_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 14799
test reply unique vocab count is: 111945


In [45]:
# sorted(unknown_text.items(), key=lambda d: d[1], reverse=True) 

### Transform more words

In [46]:
more_apostrophes = {'cannot': "can not", 'gonna': "go to", 'wanna': "want to", 'coronavirus': "COVID", 'wanted': "want", 'weeks': "week", 'feeling': "feel", 'says': "say", 'yourself': "your self", 'saying': "say", 'says': "say", 'GIF': "gif", 'waiting': "wait", 'Covid': "COVID", 'hugs': "hug", 'gave': "give", 'COVID19': "COVID", 'installing': "install", 'wants': "want", 'knows': "know", 'describes': "describe", 'following': "follow", 'asked': "ask", 'amazing': "amaze", 'finally': "final", 'minutes': "minute", 'died': "die", 'tired': "tire", 'quickly': "quick", 'gotta': "go to", 'deaths': "death", 'means': "mean", 'took': "take", 'feels': "feel", 'fans': "fan", 'numbers': "number", 'lives': "live", 'safely': "safe", 'tried': "try", 'businesses': "business", '2nd': "second", 'decided': "decide", '3rd': "third", 'hates': "hate", 'dont': "do not", 'lonely': "lone", 'totally': "total", 'excited': "excite", 'BREAKING': "break", 'gifs': "gif", 'goes': "go", 'thoughts': "thought", 'campaigning': "campaign", 'immediately': "immediate", 'teammates': "team mate", 'knew': "know", 'politicians': "politician", 'distancing': "distance", 'reopening': "reopen", 'pls': "please", 'AGAIN': "again", 'tears': "tear", 'supposed': "suppose", 'loved': "love", 'ppl': "people", 'drinking': "drink", 'Guidelines': "guide line", 'losing': "lose", 'Conference': "conference", 'officially': "official", 'OPENING': "open", 'buying': "buy", 'Gif': "gif", 'looks': "look", 'bought': "buy", 'likes': "like", 'truely': "true", 'happened': "happen", 'putting': "put", 'families': "family", 'moved': "move", 'Raise': "raise", 'helped': "help", 'vibes': "vibe", 'voting': "vote", 'showed': "show", 'Instagram': "instagram", 'spent': "spend", 'watched': "watch", 'kinda': "kind of", 'Governor': "governor", 'Coronavirus': "COVID", 'lmao': "laugh", 'seems': "seem", 'staying': "stay", 'listening': "listen", 'accounts': "account"}
def change_punc(text):
    for key in more_apostrophes.keys():
        text = text.replace(key, more_apostrophes[key])
    return text

In [47]:
df_train['map_more_punc_text'] = df_train.map_punc_text.apply(change_punc)
df_train['map_more_punc_reply'] = df_train.map_punc_reply.apply(change_punc)

In [48]:
df_dev['map_more_punc_text'] = df_dev.map_punc_text.apply(change_punc)
df_dev['map_more_punc_reply'] = df_dev.map_punc_reply.apply(change_punc)

In [49]:
df_test['map_more_punc_text'] = df_test.map_punc_text.apply(change_punc)
df_test['map_more_punc_reply'] = df_test.map_punc_reply.apply(change_punc)

In [50]:
train_text_vocab = get_vocab(df_train['map_more_punc_text'].values)
train_reply_vocab = get_vocab(df_train['map_more_punc_reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
print()

# unknown_text = check_coverage(train_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 56372
train reply unique vocab count is: 155996



In [51]:
dev_text_vocab = get_vocab(df_dev['map_more_punc_text'].values)
dev_reply_vocab = get_vocab(df_dev['map_more_punc_reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
# unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 24290
dev reply unique vocab count is: 58589


In [52]:
test_text_vocab = get_vocab(df_test['map_more_punc_text'].values)
test_reply_vocab = get_vocab(df_test['map_more_punc_reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
# unknown_text = check_coverage(test_text_vocab, roberta_vocab)
# print()
# unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 14715
test reply unique vocab count is: 111826


In [53]:
# sorted(unknown_text.items(), key=lambda d: d[1], reverse=True) 

## Store preprocessed results

In [54]:
df_train.head()

,idx,text,categories,context_idx,reply,mp4,label,map_punc_text,map_punc_reply,map_more_punc_text,map_more_punc_reply
0,8981,How incredibly despicable these traitors are....,[],0,@JosephJFlynn1 @GenFlynn @BarbaraRedgate @GoJa...,,fake,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...
1,8981,How incredibly despicable these traitors are....,[],1,@JosephJFlynn1 @GenFlynn @BarbaraRedgate @GoJa...,,fake,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...
2,8981,How incredibly despicable these traitors are....,[],2,@JosephJFlynn1 @GenFlynn @BarbaraRedgate @GoJa...,,fake,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...
3,8981,How incredibly despicable these traitors are....,[],3,@JosephJFlynn1 @Dean_Moriarty36 @GenFlynn @Bar...,,fake,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ Dean - Moriarty36 @ Gen...,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ Dean - Moriarty36 @ Gen...
4,8981,How incredibly despicable these traitors are....,[],5,@JosephJFlynn1 @kldreams61 @GenFlynn @BarbaraR...,,fake,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ kldreams61 @ GenFlynn @ ...,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ kldreams61 @ GenFlynn @ ...


In [55]:
df_dev.head()

,idx,text,categories,context_idx,reply,mp4,map_punc_text,map_punc_reply,map_more_punc_text,map_more_punc_reply
0,0,If there was any moment that I needed love and...,[hug],0,"Hey, Court!! If you need anything, you know i ...",7d21ea02003e5574293e10c2583fe415.mp4,If there was any moment that I needed love and...,"Hey , Court ! ! If you need anything , you...",If there was any moment that I needed love and...,"Hey , Court ! ! If you need anything , you..."
1,1,Hi Guys up for a chat ..... if yes see you at ...,[dance],0,Yeahh..... We are Waiting\n#AskSid,6cf1ca01c8ea9513f3e355e2d56d3dd4.mp4,Hi Guys up for a chat . . . . . if yes s...,Yeahh . . . . . We are Waiting\n # AskSid,Hi Guys up for a chat . . . . . if yes s...,Yeahh . . . . . We are Waiting\n # AskSid
2,2,Did I get myself physically stuck in a filing ...,[no],0,,7e86aaeb736f91d36b56684ce1199602.mp4,Did I get myself physically stuck in a filing ...,,Did I get myself physically stuck in a filing ...,
3,3,Day one on medication and I no longer have dep...,[applause],0,,ba1e36cbdb03043a858f1d40e1bb2420.mp4,Day one on medication and I no longer have dep...,,Day one on medication and I no longer have dep...,
4,4,"Drop your cash app, use hashtag #BailoutHumansNow","[applause, dance, happy_dance]",0,$tyratomaro #BailoutHumans,096e67b87796d76f363d62d61f952ea3.mp4,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans


In [56]:
df_test.head()

,idx,text,categories,context_idx,reply,mp4,map_punc_text,map_punc_reply,map_more_punc_text,map_more_punc_reply
0,44000,If a passerby had not posted #HillaryClinton f...,[],0,@RealJamesWoods @CNN Trump Warns Iranian Presi...,,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Trump Warns Iranian P...,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Trump Warns Iranian P...
1,44000,If a passerby had not posted #HillaryClinton f...,[],1,@RealJamesWoods @CNN Why Did 'Russian Agent' B...,,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Why Did ' Russian ...,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Why Did ' Russian ...
2,44000,If a passerby had not posted #HillaryClinton f...,[],2,@RealJamesWoods @CNN This is nothing but truth.,,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN This is nothing but t...,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN This is nothing but t...
3,44000,If a passerby had not posted #HillaryClinton f...,[],3,@RealJamesWoods @CNN Good to see you @RealJame...,,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Good to see you @ Re...,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Good to see you @ Re...
4,44000,If a passerby had not posted #HillaryClinton f...,[],4,@RealJamesWoods @CNN Israeli Jets Strike Syria...,,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Israeli Jets Strike S...,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Israeli Jets Strike S...


## Output preprocessed to json

In [58]:
df_preprocessed = df_train[['idx', 'map_more_punc_text', 'map_more_punc_reply', 'label']].copy()
df_preprocessed.columns = ['idx', 'text', 'reply', 'label']
df_preprocessed.to_json('evaluation/source/preprocess_train.json', orient='records', lines=True)

In [38]:
df_train.head()

,idx,text,categories,context_idx,reply,mp4,label
0,8981,How incredibly despicable these traitors are....,[],0,@JosephJFlynn1 @GenFlynn @BarbaraRedgate @GoJa...,,fake
1,8981,How incredibly despicable these traitors are....,[],1,@JosephJFlynn1 @GenFlynn @BarbaraRedgate @GoJa...,,fake
2,8981,How incredibly despicable these traitors are....,[],2,@JosephJFlynn1 @GenFlynn @BarbaraRedgate @GoJa...,,fake
3,8981,How incredibly despicable these traitors are....,[],3,@JosephJFlynn1 @Dean_Moriarty36 @GenFlynn @Bar...,,fake
4,8981,How incredibly despicable these traitors are....,[],5,@JosephJFlynn1 @kldreams61 @GenFlynn @BarbaraR...,,fake


In [59]:
df_preprocessed_dev = df_dev[['idx', 'map_more_punc_text', 'map_more_punc_reply']].copy()
df_preprocessed_dev.columns = ['idx', 'text', 'reply']
df_preprocessed_dev.to_json('evaluation/source/preprocess_dev.json', orient='records', lines=True)

In [60]:
df_preprocessed_test = df_test[['idx', 'map_more_punc_text', 'map_more_punc_reply']].copy()
df_preprocessed_test.columns = ['idx', 'text', 'reply']
df_preprocessed_test.to_json('evaluation/source/preprocess_test.json', orient='records', lines=True)